In [ ]:
%pip install scikit-learn google-cloud-aiplatform

## Setup GCP

In [ ]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "key.json"

In [ ]:
from vertexai.resources.preview import FeatureOnlineStore

# my_fos = FeatureOnlineStore.create_optimized_store(
#     'test',
#     True,
#     ["mlops-433612"],
# )

test_fos = FeatureOnlineStore('projects/994601067814/locations/us-central1/featureOnlineStores/test')
test_fos.gca_resource

In [ ]:
from vertexai.resources.preview import FeatureViewBigQuerySource, FeatureView

my_fv = test_fos.create_feature_view(
    'test_fv',
    FeatureViewBigQuerySource(
        uri=f"bq://mlops-433612.titanic_survival.passenger",
        entity_id_columns=["Name","Pclass","Survived"],
    ),
    # sync_config can be set with CRON_SCHEDULE if you want to setup auto sync
    sync_config=None,
)

In [ ]:
feature_view = FeatureView('projects/994601067814/locations/us-central1/featureOnlineStores/test/featureViews/test_fv')
fv_sync = feature_view.sync()


import time

while True:
    feature_view_sync = my_fv.get_sync(fv_sync.resource_name.split("/")[9]).gca_resource
    if feature_view_sync.run_time.end_time.seconds > 0:
        status = "Succeed" if feature_view_sync.final_status.code == 0 else "Failed"
        print(f"Sync {status} for {feature_view_sync.name}.")
        # wait a little more for the job to properly shutdown
        time.sleep(30)
        break
    else:
        print("Sync ongoing, waiting for 30 seconds.")
    time.sleep(30)

In [ ]:
from google.cloud.aiplatform_v1beta1 import FeatureOnlineStoreAdminServiceClient

LOCATION = "us-central1"  # @param {type:"string"}

API_ENDPOINT = f"{LOCATION}-aiplatform.googleapis.com"

admin_client = FeatureOnlineStoreAdminServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

PROJECT_ID = "mlops-433612"

FEATURE_ONLINE_STORE_ID = "test"

admin_client.get_feature_online_store(
    name=f"projects/{PROJECT_ID}/locations/{LOCATION}/featureOnlineStores/{FEATURE_ONLINE_STORE_ID}"
)

## Public Endpoint

In [ ]:
import requests
import subprocess

# Step 1: Get the access token using gcloud command
access_token = subprocess.check_output(["gcloud", "auth", "print-access-token"]).decode("utf-8").strip()

# Step 2: Define the necessary parameters
PROJECT_ID = "mlops-433612"
LOCATION_ID = "us-central1"
FEATUREONLINESTORE_NAME = "titanic_survival_passengers"
FEATUREVIEW_NAME = "titanic_survival_passengers"

# Construct the URL
url = f"https://{LOCATION_ID}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{LOCATION_ID}/featureOnlineStores/{FEATUREONLINESTORE_NAME}"

# Step 3: Set up the headers with the authorization token
headers = {
    "Authorization": f"Bearer {access_token}"
}

# Step 4: Make the GET request
response = requests.get(url, headers=headers)

# Step 5: Check the response status and output the content
if response.status_code == 200:
    print("Request was successful.")
    print(response.json())
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")
    print(response.text)


## Fetch Data From Vertex AI Feature View

In [ ]:
from google.cloud.aiplatform_v1 import FeatureOnlineStoreServiceClient
from google.cloud.aiplatform_v1.types import feature_online_store_service as feature_online_store_service_pb2

PUBLIC_ENDPOINT_DOMAIN_NAME = "us-central1-aiplatform.googleapis.com"
PROJECT_ID = "mlops-433612"
LOCATION_ID = "us-central1"
FEATUREONLINESTORE_NAME = "titanic_survival_passengers"
FEATUREVIEW_NAME = "titanic_survival_passengers"

data_client = FeatureOnlineStoreServiceClient(
  client_options={"api_endpoint": PUBLIC_ENDPOINT_DOMAIN_NAME}
)

response = data_client.fetch_feature_values(
  request=feature_online_store_service_pb2.FetchFeatureValuesRequest(
    feature_view=f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/featureOnlineStores/{FEATUREONLINESTORE_NAME}/featureViews/{FEATUREVIEW_NAME}",
  ),
)